In [ ]:
import torch
 
torch.cuda.get_device_name(device=None)

'Tesla P100-PCIE-16GB'

In [ ]:
!nvidia-smi

# Train Yolo v5 for bosch Hackathon 
## With default weights of yolov5m6.pt - YOLO v5 P6 m sized model
~ Lakshya

---

#CLONING THE YOLO-V5 FILES FROM CUSTOM REPOSITORY

In [ ]:
!git clone https://github.com/lakshyads/yolov5-base-custom.git yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 112 (delta 20), reused 111 (delta 19), pack-reused 0
Receiving objects: 100% (112/112), 1.11 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (20/20), done.


# CHANGING THE DIRECTORY

In [ ]:
%cd yolov5/

/content/yolov5


#INSTALLING THE DEPENDENCIES

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 645kB 6.8MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cd ..


/content


## READY THE DATASET


In [ ]:
!pip install PyDrive

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Download zipped data to Colab from Google Drive

In [ ]:
# File id for train-validate-test-split-data.zip = 1S23A8_uuiE2zp50qJDSp_xcC294kaFW7
download = drive.CreateFile({'id': '1S23A8_uuiE2zp50qJDSp_xcC294kaFW7'})
download.GetContentFile('train-validate-test-split-data.zip')

## Download previously saved weights


In [ ]:
# Folder id on GDrive for backups
# download = drive.CreateFile({'id': '1Fo9h4-e-AvGSeYX0Q_086MnP7uquJMyN'})
# download.GetContentFile('backups')

### Unzip in this workspace

In [ ]:
!unzip -q /content/train-validate-test-split-data.zip 

## Change the directory again after installing dependencied if restart runtime required

In [ ]:
%cd yolov5/

/content/yolov5


## Give full permissions to everthing inside /content/yolov5

In [ ]:
!chmod -R 755 /content/yolov5


# DOWNLOADING ALL VERSIONS OF YOLO-V5 MODEL

In [ ]:
!/content/yolov5/weights/download_weights.sh

100% 14.1M/14.1M [00:00<00:00, 43.3MB/s]

100% 41.1M/41.1M [00:00<00:00, 82.9MB/s]

100% 90.2M/90.2M [00:00<00:00, 102MB/s] 

100% 168M/168M [00:01<00:00, 106MB/s] 



# STEPS BEFORE TRAINING CUSTOM DATASET:
1. Go to yolov5/data/
2. Open coco128.yaml
3. Edit the following inside it:
    

      1.   Training and Test file path
      2.   Number of classes and Class names.



# TRAINING THE YOLO-V5 MODEL

In [ ]:
!python train.py --img 640 --batch 16 --epochs 150 --data my_data.yaml --weights yolov5s6.pt --nosave --cache 

## Upload Best weights to GDrive after training 
### Also Zip and upload complete runs folder to GDrive as precaution

In [ ]:
from datetime import datetime
import shutil
import os

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d%m%Y-T-%H%M%S")
model_name = 'yolov5s6'
epocs = '150'
batch_size = '16'
initial_weights = 'pre_Init_Weights'
# initial_weights = 'custom_Init_Weights'


In [ ]:
uploadFileName = f'weights-{model_name}-{initial_weights}-epocs_{epocs}-batch_{batch_size}-{dt_string}.txt'
try:
  print('\n Trying to upload best weights to gdrive ...')
  upload = drive.CreateFile({'title': uploadFileName, 'parents': [{'id': '1Fo9h4-e-AvGSeYX0Q_086MnP7uquJMyN'}]})
  upload.SetContentFile('/content/yolov5/runs/train/exp/weights/last.pt')
  upload.Upload()
  print(f'\n Best weights uploaded to google drive. Uploaded file name = {uploadFileName}')
except Exception as e:
  print(f'\n Failed to upload best weight file: {uploadFileName} to Google Drive. \nException details: {e}')


 Trying to upload best weights to gdrive ...

 Best weights uploaded to google drive. Uploaded file name = weights-yolov5s6-pre_Init_Weights-epocs_150-batch_16-03062021-T-182444.txt


In [ ]:
bak_archive_name = f'run-{model_name}-{initial_weights}-epocs_{epocs}-batch_{batch_size}-{dt_string}'
bak_archive_path = r'/content/yolov5/backups-for-gdrive/'
bak_archive = os.path.join(bak_archive_path, bak_archive_name)
dir_to_archive = '/content/yolov5/runs'
try:
  print('\n Trying to upload run folder to gdrive ...')
  if not os.path.exists(bak_archive_path):
    os.makedirs(bak_archive_path)

  shutil.make_archive(base_name=bak_archive, root_dir=dir_to_archive, format= 'zip', base_dir=dir_to_archive)

  file = drive.CreateFile({'title': f'{bak_archive_name}.zip', 'parents': [{'id': '1Fo9h4-e-AvGSeYX0Q_086MnP7uquJMyN'}]})
  file.SetContentFile(f'{bak_archive}.zip')
  file.Upload()  
  print(f'\n Run data uploaded to google drive. Uploaded file name = {bak_archive}')
except Exception as e:
  print(f'\n Failed to upload Run data to GDrive: {bak_archive} to Google Drive. \nException details: {e}')


 Trying to upload run folder to gdrive ...

 Run data uploaded to google drive. Uploaded file name = /content/yolov5/backups-for-gdrive/run-yolov5s6-pre_Init_Weights-epocs_150-batch_16-03062021-T-182444


# VISUALIZING THE TRAINING METRICS

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

## PLOTTING TRAINING RESULTS

In [ ]:
from utils.plots import plot_results

plot_results(save_dir='runs/train/exp3')

# TESTING

In [ ]:
!python test.py --weights /content/yolov5/runs/train/exp3/weights/best.pt --data my_data.yaml --img 640

## Extract new images from zipped upload


In [ ]:
%cd ..
!unzip -q /content/extracted_images.zip 
%cd yolov5

/content
/content/yolov5


# DETECTING ON NEW IMAGES

In [ ]:
!python detect.py --source /content/extracted_images --weights /content/last.pt --img 640 --save-txt --save-conf

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=True, save_crop=False, save_txt=True, source='/content/extracted_images', update=False, view_img=False, weights=['/content/last.pt'])
YOLOv5 🚀 a173e41 torch 1.8.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 291 layers, 12360844 parameters, 0 gradients, 16.7 GFLOPS
image 1/46 /content/extracted_images/frame1.jpg: 640x384 Done. (0.020s)
image 2/46 /content/extracted_images/frame10.jpg: 640x384 1 Furniture, Done. (0.020s)
image 3/46 /content/extracted_images/frame11.jpg: 640x384 1 Door, Done. (0.014s)
image 4/46 /content/extracted_images/frame12.jpg: 640x384 1 Furniture, Done. (0.014s)
image 5/46 /content/extracted_images/frame13.jpg: 640x384 1 Door, Done. (0.014s)
image 6/46 /content/extracted_

#RETRAINING FROM THE LAST CHECKPOINT

In [ ]:
!python train.py --weights /content/yolov5/runs/train/exp2/weights/last.pt --epochs 100 --img 416

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
path='/content/yolov5/runs/detect/exp3/labels/11_image.txt'
myfile=open(path,'r')
lines=myfile.readlines()
pattern= "class_name"

cordinate = []
for line in lines:
  for x in line.splitlines():
    cordinate.append([x[1:]])
#   if re.search(pattern,line):
#     Cord_Raw=line
# Cord=Cord_Raw.split("(")[1].split(")")[0].split("  ")

In [ ]:
cordinate

In [ ]:
cordinate[0]

In [ ]:

import cv2
img = cv2.imread("/content/drive/MyDrive/images/train/11_image.jpg")
print(img.shape)
for i in cordinate:
  count= 0
  for val in i:
    value = val.strip().split(" ")
    x_min = int(float(value[0])*img.shape[0])
    x_max = x_min + int(float(value[2])*img.shape[0])
    y_min = int(float(value[1])*img.shape[1])
    y_max = y_min + int(float(value[3])*img.shape[1])
    crop_img = img[y_min:y_max, x_min:x_max]
    cv2.imwrite(f"Object_{count}.jpg",crop_img)
    count = count + 1





In [ ]:
import cv2
img = cv2.imread("/content/drive/MyDrive/images/train/11_image.jpg")
crop_img = img[y_min:y_max, x_min:x_max]
cv2.imwrite("Object.jpg",crop_img)